In [26]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [27]:
# generate master problem
# return dual value to do the subproblem
def master_problem(cut_pattern):
    global number_iteration
    m = gp.Model("Master Problem")
    x = m.addMVar(shape = cut_pattern.shape[1], lb=0)
    m.addConstr(cut_pattern @ x >= number_demand)
    m.setObjective(x.sum(), GRB.MINIMIZE)
    m.Params.OutputFlag = 0
    m.optimize()

    number_iteration += 1
    print('iteration {} \nRestricted master problem:  Required number of patterns: {}    X: {}'.format(number_iteration, round(m.objVal,2), np.around(m.getAttr('X'),2)))
    return np.array(m.getAttr('pi', m.getConstrs()))

In [28]:
# solve the subproblem to determine whether continue to generate column
# return determine and new column
def knapsack_subproblem(shadow_price):
    m = gp.Model("subproblem")
    x = m.addMVar(shape = shadow_price.shape[0],lb=0, vtype = GRB.INTEGER)
    m.addConstr(lengths @ x <= W)
    m.setObjective(1-shadow_price @ x, GRB.MINIMIZE)
    m.Params.OutputFlag = 0
    m.optimize()

    determine = m.objVal < 0
    # for a minimize problem, if min{cj-zj>=0}, it stops generating new column, otherwise it generate new pattern
    if determine:
        new_column = m.getAttr('X', m.getVars())
    else:
        new_column = None
    print("subproblem:    cj-zj: {}  new pattern: {}".format(round(m.objVal,2), m.getAttr("X")))
    print()
    return determine, new_column

In [29]:
# Retrieve related parameters from .mps file
def get_parameters(file_name):
    m = gp.read(file_name)
    demands = []
    for i in m.getConstrs():
        demands.append(i.RHS)
    W = demands.pop(-1)
    
    lengths = []
    for i in m.getVars():
        lengths.append(i.obj)
    lengths.pop(-1)

    W = np.array(W)
    lengths = np.array(lengths)
    demands = np.array(demands)

    return W, lengths, demands

In [30]:
# generate the parameters of initial master problem
number_iteration = 0
W, lengths, number_demand = get_parameters("param.mps")
print("Total length of each wood: ", W)
print("Demands for different lengths ",number_demand)
print("Required length ", lengths)
print()

# initialize pattern
cut_pattern = []
for j in lengths:
    cut_pattern.append(W//j)
cut_pattern = np.array(cut_pattern)
cut_pattern = np.diag(cut_pattern)

determine = True
new_cut_pattern = None

while determine:
    if new_cut_pattern:
        cut_pattern = np.column_stack((cut_pattern, new_cut_pattern))
    shadow_price = master_problem(cut_pattern)
    determine, new_cut_pattern = knapsack_subproblem(shadow_price)

print(f'cut_pattern: {cut_pattern}')


Read MPS format model from file cutting_stock_model.mps
Reading time = 0.00 seconds
cs: 11 rows, 11 columns, 11 nonzeros
Total length of each wood:  20.0
Demands for different lengths  [10. 20. 30. 10. 20. 30. 10. 20. 10. 20.]
Required length  [ 1.  2.  3.  4.  7.  8. 10. 13. 14. 16.]

iteration 1 
Restricted master problem:  Required number of patterns: 89.5    X: [ 0.5  2.   5.   2.  10.  15.   5.  20.  10.  20. ]
subproblem:    cj-zj: -0.5  new pattern: [0.0, -0.0, 0.0, -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, -0.0]

iteration 2 
Restricted master problem:  Required number of patterns: 79.5    X: [ 0.5  2.   5.   2.   0.  15.   5.   0.  10.  20.  20. ]
subproblem:    cj-zj: -0.38  new pattern: [1.0, -0.0, 2.0, -0.0, -0.0, 0.0, -0.0, 1.0, -0.0, -0.0]

iteration 3 
Restricted master problem:  Required number of patterns: 79.5    X: [ 0.5  2.   5.   2.   0.  15.   5.   0.  10.  20.  20.   0. ]
subproblem:    cj-zj: -0.33  new pattern: [0.0, -0.0, 2.0, 0.0, 0.0, 0.0, -0.0, 0.0, 1.0, -0.0]

ite